In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lcc-fasd-casia-combined/lcc-fasd-casia/LCC_FASD/LCC_FASD_evaluation/spoof/spoof_5071.png
/kaggle/input/lcc-fasd-casia-combined/lcc-fasd-casia/LCC_FASD/LCC_FASD_evaluation/spoof/spoof_5031.png
/kaggle/input/lcc-fasd-casia-combined/lcc-fasd-casia/LCC_FASD/LCC_FASD_evaluation/spoof/spoof_314.png
/kaggle/input/lcc-fasd-casia-combined/lcc-fasd-casia/LCC_FASD/LCC_FASD_evaluation/spoof/spoof_2550.png
/kaggle/input/lcc-fasd-casia-combined/lcc-fasd-casia/LCC_FASD/LCC_FASD_evaluation/spoof/spoof_2234.png
/kaggle/input/lcc-fasd-casia-combined/lcc-fasd-casia/LCC_FASD/LCC_FASD_evaluation/spoof/spoof_3188.png
/kaggle/input/lcc-fasd-casia-combined/lcc-fasd-casia/LCC_FASD/LCC_FASD_evaluation/spoof/spoof_1324.png
/kaggle/input/lcc-fasd-casia-combined/lcc-fasd-casia/LCC_FASD/LCC_FASD_evaluation/spoof/spoof_1590.png
/kaggle/input/lcc-fasd-casia-combined/lcc-fasd-casia/LCC_FASD/LCC_FASD_evaluation/spoof/spoof_1363.png
/kaggle/input/lcc-fasd-casia-combined/lcc-fasd-casia/LCC_FASD/LCC_FASD_eva

In [2]:
import torch
import torchvision
from torchvision import datasets
from torchvision import transforms as T # for simplifying the transforms
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, sampler, random_split
from torchvision import models

In [3]:
# remove warnings
import warnings
warnings.filterwarnings("ignore")

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
import sys
from tqdm import tqdm
import time
import copy

In [6]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [7]:
def get_classes(data_dir):
    all_data = datasets.ImageFolder(data_dir)
    return all_data.classes

In [8]:
data_dir = '/kaggle/input/lcc-fasd-casia-combined/lcc-fasd-casia/LCC_FASD'
batch_size = 32

In [9]:
def get_swin_transformer_data_loaders(data_dir, batch_size, train=True, image_size=(224, 224)):
    if train:
        # Train data transformations
        transform = transforms.Compose([
            transforms.RandomResizedCrop(image_size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
        ])
        dataset = datasets.ImageFolder(os.path.join(data_dir, "LCC_FASD_training/"), transform=transform)
        data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)
        return data_loader, len(dataset)
    else:
        # Validation/test data transformations
        transform = transforms.Compose([
            transforms.Resize(image_size),
            transforms.CenterCrop(image_size),
            transforms.ToTensor(),
        ])
        val_dataset = datasets.ImageFolder(os.path.join(data_dir, "LCC_FASD_development/"), transform=transform)
        test_dataset = datasets.ImageFolder(os.path.join(data_dir, "LCC_FASD_evaluation/"), transform=transform)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
        return val_loader, test_loader, len(val_dataset), len(test_dataset)

(train_loader, train_data_len) = get_swin_transformer_data_loaders(data_dir, batch_size, train=True)
(val_loader, test_loader, val_data_len, test_data_len) = get_swin_transformer_data_loaders(data_dir, batch_size, train=False)


In [10]:

dataset_path = "/kaggle/input/lcc-fasd-casia-combined"

In [11]:
classes = get_classes("/kaggle/input/lcc-fasd-casia-combined/lcc-fasd-casia/LCC_FASD/LCC_FASD_training")
print(classes, len(classes))

['real', 'spoof'] 2


In [12]:
classes = get_classes("/kaggle/input/lcc-fasd-casia-combined/lcc-fasd-casia/LCC_FASD/LCC_FASD_evaluation")
print(classes, len(classes))

['real', 'spoof'] 2


In [13]:
classes = get_classes("/kaggle/input/lcc-fasd-casia-combined/lcc-fasd-casia/LCC_FASD/LCC_FASD_development")
print(classes, len(classes))

['real', 'spoof'] 2


In [14]:
dataloaders = {
    "train": train_loader,
    "val": val_loader
}
dataset_sizes = {
    "train": train_data_len,
    "val": val_data_len
}

In [15]:
print(len(train_loader), len(val_loader), len(test_loader))

274 94 239


In [16]:
print(train_data_len, val_data_len, test_data_len)

8746 3006 7635


In [17]:
# now, for the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [18]:
HUB_URL = "SharanSMenon/swin-transformer-hub:main"
MODEL_NAME = "swin_tiny_patch4_window7_224"
# check hubconf for more models.
model = torch.hub.load(HUB_URL, MODEL_NAME, pretrained=True) # load from torch hub

Downloading: "https://github.com/SharanSMenon/swin-transformer-hub/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_tiny_patch4_window7_224.pth" to /root/.cache/torch/hub/checkpoints/swin_tiny_patch4_window7_224.pth
100%|██████████| 109M/109M [00:00<00:00, 241MB/s] 


In [19]:

for param in model.parameters(): #freeze model
    param.requires_grad = False

n_inputs = model.head.in_features
model.head = nn.Sequential(
    nn.Linear(n_inputs, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, len(classes))
)
model = model.to(device)
print(model.head)

Sequential(
  (0): Linear(in_features=768, out_features=512, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.3, inplace=False)
  (3): Linear(in_features=512, out_features=2, bias=True)
)


In [20]:
## Now, we import timm, torchvision image models
!pip install timm # kaggle doesnt have it installed by default
import timm
from timm.loss import LabelSmoothingCrossEntropy # This is better than normal nn.CrossEntropyLoss

In [21]:
criterion = LabelSmoothingCrossEntropy()
criterion = criterion.to(device)
optimizer = optim.AdamW(model.head.parameters(), lr=0.005)

In [22]:
# lr scheduler
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.97)

In [23]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print("-"*10)

        for phase in ['train', 'val']: # We do training and validation phase per epoch
            if phase == 'train':
                model.train() # model to training mode
            else:
                model.eval() # model to evaluate

            running_loss = 0.0
            running_corrects = 0.0

            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'): # no autograd makes validation go faster
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1) # used for accuracy
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step() # step at end of epoch

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc =  running_corrects.double() / dataset_sizes[phase]
            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict()) # keep the best validation accuracy model
        print()
    time_elapsed = time.time() - since # slight error
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print("Best Val Acc: {:.4f}".format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

In [24]:
model_fit = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=400) # now it is a lot faster

Epoch 0/399
----------


100%|██████████| 274/274 [01:09<00:00,  3.96it/s]


train Loss: 0.3769 Acc: 0.8958


100%|██████████| 94/94 [00:24<00:00,  3.78it/s]


val Loss: 0.3128 Acc: 0.9401

Epoch 1/399
----------


...

val Loss: 0.2612 Acc: 0.9714

Epoch 399/399
----------


100%|██████████| 274/274 [01:00<00:00,  4.50it/s]


train Loss: 0.2456 Acc: 0.9788


100%|██████████| 94/94 [00:25<00:00,  3.64it/s]

val Loss: 0.2611 Acc: 0.9714

Training complete in 556m 38s
Best Val Acc: 0.9784


In [25]:
test_loss = 0.0
class_correct = list(0 for i in range(len(classes)))
class_total = list(0 for i in range(len(classes)))
model_fit.eval()

for data, target in tqdm(test_loader):
    data, target = data.to(device), target.to(device)
    with torch.no_grad(): # turn off autograd for faster testing
        output = model_fit(data)
        loss = criterion(output, target)
    test_loss = loss.item() * data.size(0)
    _, pred = torch.max(output, 1)
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    if len(target) == 32:
        for i in range(32):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1

test_loss = test_loss / test_data_len
print('Test Loss: {:.4f}'.format(test_loss))
for i in range(len(classes)):
    if class_total[i] > 0:
        print("Test Accuracy of %5s: %2d%% (%2d/%2d)" % (
            classes[i], 100*class_correct[i]/class_total[i], np.sum(class_correct[i]), np.sum(class_total[i])
        ))
    else:
        print("Test accuracy of %5s: NA" % (classes[i]))
print("Test Accuracy of %2d%% (%2d/%2d)" % (
            100*np.sum(class_correct)/np.sum(class_total), np.sum(class_correct), np.sum(class_total)
        ))

100%|██████████| 239/239 [01:02<00:00,  3.81it/s]

Test Loss: 0.0005
Test Accuracy of  real: 73% (239/323)
Test Accuracy of spoof: 97% (7138/7293)
Test Accuracy of 96% (7377/7616)
